In [1]:
import re
# Numpy 
import numpy as np
# Pickle
import pickle
# Pandas
import pandas as pd
# Hugging Face
import huggingface_hub
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# PyTorch
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# SkLearn
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [2]:
# Load datasets 
# Hate Xplain
hate_xplain = pd.read_csv("project/data/hate_xplain.csv")

# Implicit Hate 
implicit_hate = pd.read_csv('project/data/implicit-hate-corpus/implicit_hate_v1_stg2_posts.tsv', delimiter='\t')
label_map = {
    'white_grievance': 0, 'incitement': 1, 'inferiority': 2,
    'irony': 3, 'stereotypical': 4, 'threatening': 5, 'other': 6
}

implicit_hate['class_label'] = implicit_hate['implicit_class'].map(label_map)
implicit_hate.drop("extra_implicit_class", axis=1, inplace=True)

# Toxic-Spans
annotations = pd.read_csv('project/data/toxic-spans/annotations.csv')
comments = pd.read_csv('project/data/toxic-spans/comments.csv')

toxic_spans = pd.merge(annotations, comments, on='comment_id')

In [3]:
bert = 'distilbert-base-uncased'
bert_model = AutoModelForSequenceClassification.from_pretrained(bert, problem_type="multi_label_classification", num_labels=3, 
                id2label={0: "hate speech", 1: "normal", 2: "offensive"}, label2id={"hate speech": 0, "normal": 1, "offensive": 2}, max_length=128)
tokenizer = AutoTokenizer.from_pretrained(bert)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def tokenize_texts(texts, tokenizer, max_length):
    texts = texts.to_list()
    encodings = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    return encodings

texts = hate_xplain['tweet']
hx_enc = tokenize_texts(texts, tokenizer, 128)


In [8]:
def get_embeddings(model, encodings):
    with torch.no_grad():
        outputs = model(**encodings)
        token_embeddings = outputs.last_hidden_state
        sentence_embeddings = token_embeddings.mean(dim=1) 
    return sentence_embeddings

hx_bert_embed = get_embeddings(bert_model, hx_enc)

: 

: 